In [1]:
from bs4 import BeautifulSoup
import re
import string

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
punctuations = '''`=«»،÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             ّ    | # shadda
                             ٰ    | #qasr
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ۛ    |##
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

def remove_digits(text):
    text = re.sub(r"[1234567890١٢٣٤٥٦٧٨٩٠]+", "", text)
    return text

def remove_english_characters(text):
    text = re.sub(r'[a-zA-Z]+','',text)
    return text
    

def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

def normalization(sentence):
    text = remove_english_characters(sentence)
    text = remove_digits(text)
    text = remove_diacritics(text)
    text = remove_punctuations(text)        
    return text 

creating a text file from raw quran translation

In [ ]:
with open('/content/drive/MyDrive/makarem.xml', 'r', encoding="utf8") as f:
    content = f.read() 

soup= BeautifulSoup(content, 'xml')

verses = soup.find_all('s')

with open('/content/drive/MyDrive/quran.txt' , 'w') as f:
  for verse in verses:
    normal_verse = normalization(verse.contents[0])
    print(normal_verse , file=f)

In [ ]:
pip install transformers datasets sentencepiece

# Further pretrain ParsBERT model

DO NOT RUN THIS CELL

In [ ]:
from transformers import BertTokenizer, BertForMaskedLM
model_name  = "HooshvareLab/bert-fa-zwnj-base"

tokenizer = BertTokenizer.from_pretrained(model_name)
# model_path= "/content/drive/MyDrive/Quran_QA/quranBERT/checkpoint-6236"
# model = BertForMaskedLM.from_pretrained(model_path)

model = BertForMaskedLM.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')

In [5]:
from transformers import BertTokenizer, BertForMaskedLM

# v3.0
model_name_or_path = "HooshvareLab/bert-fa-zwnj-base"

tokenizer = BertTokenizer.from_pretrained(model_name_or_path)

# model = TFAutoModel.from_pretrained(model_name_or_path)  For TF
model = BertForMaskedLM.from_pretrained(model_name_or_path)

In [6]:
quranpath = '/content/drive/MyDrive/quran.txt'
with open(quranpath , 'r' , encoding= 'utf-8') as f:
    lines = f.readlines()

In [7]:
for i in range(len(lines)):
  lines[i] = lines[i][:-1]

In [8]:
import numpy as np

verse_lengths = [len(x.split()) for x in lines]
maxlength =  max(verse_lengths)
maxlengthid =  np.argmax([verse_lengths])
verse_lengths = sorted([(i , x) for i , x in enumerate(verse_lengths)] ,key =lambda x : x[1] )
maxlength

301

In [9]:
from datasets import load_dataset
dataset = load_dataset('text', data_files={'train': [quranpath], 'test':quranpath})

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-a2850252c4ff5c38/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 6236
    })
    test: Dataset({
        features: ['text'],
        num_rows: 6236
    })
})

In [10]:
def tokenize_pad_and_truncate(texts):
    return tokenizer(texts["text"], padding="max_length")

tokenized_datasets = dataset.map(tokenize_pad_and_truncate, batched=True)

Map:   0%|          | 0/6236 [00:00<?, ? examples/s]

Map:   0%|          | 0/6236 [00:00<?, ? examples/s]

In [11]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm_probability=0.15)

In [12]:
from transformers import  TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Quran_QA/quranBERT_moreepochs",
    save_strategy='epoch',
    evaluation_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=4,
    weight_decay=0.01,
)

In [13]:
from random import randint
testset =tokenized_datasets['test'].select([randint(0 , 6000) for _ in range(200)])
testset

Dataset({
    features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 200
})

In [14]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset= tokenized_datasets['train'] ,
    eval_dataset = testset,
    data_collator=data_collator,
    tokenizer= tokenizer
)

In [ ]:
len(tokenized_datasets['train'])

6236

In [15]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,3.063900,nan
2,2.786600,nan
3,2.450400,nan
4,2.520100,2.238355


TrainOutput(global_step=12472, training_loss=2.745673834780533, metrics={'train_runtime': 3547.7929, 'train_samples_per_second': 7.031, 'train_steps_per_second': 3.515, 'total_flos': 6566260386299904.0, 'train_loss': 2.745673834780533, 'epoch': 4.0})

In [16]:
model.save_pretrained("/content/drive/MyDrive/Quran_QA/quranBERT/8epochs")

In [17]:
tokenizer.save_pretrained("/content/drive/MyDrive/Quran_QA/quranBERT/8epochs")

('/content/drive/MyDrive/Quran_QA/quranBERT/8epochs/tokenizer_config.json',
 '/content/drive/MyDrive/Quran_QA/quranBERT/8epochs/special_tokens_map.json',
 '/content/drive/MyDrive/Quran_QA/quranBERT/8epochs/vocab.txt',
 '/content/drive/MyDrive/Quran_QA/quranBERT/8epochs/added_tokens.json')

In [18]:
sample = tokenizer(["به‌نام خداوند بخشنده و [MASK]"] , return_tensors='pt').to("cuda")
result =model(**sample)
result

MaskedLMOutput(loss=None, logits=tensor([[[ -7.9317,  -2.3641,  -1.9318,  ...,  -5.2123,  -5.7007,  -8.8080],
         [ -9.9400,  -1.6982,  -1.4320,  ...,  -8.2713,  -7.2054, -10.5717],
         [ -5.1274,  -4.4563,  -2.5290,  ...,  -4.4256,  -2.4244,  -6.2234],
         ...,
         [ -8.6829,  -3.3124,  -3.8511,  ...,  -6.7178,  -5.4029,  -9.3553],
         [ -6.5777,  -1.5904,  -1.1403,  ...,  -4.5268,  -2.6685,  -7.2153],
         [ -7.7374,  -2.1624,  -1.7332,  ...,  -4.8981,  -5.6102,  -8.7348]]],
       device='cuda:0', grad_fn=<ViewBackward0>), hidden_states=None, attentions=None)

In [19]:
import numpy as np

ids =np.argmax(result[0].detach().cpu().numpy(), axis= 2)
print(ids)
tokenizer.decode(ids[0])

[[  623   623  8334 30508   623 11233   623]]


'و و خداوند بخشنده و مهربان و'